We import what will serve us

In [6]:
from pandas import DataFrame, read_csv
import bigml.api
from bigml.api import BigML
import kaggle


## Loading csv files as data frames

Files must be placed in the same directory as this file. Alternatively, modify the relative path to those files.

In [17]:
!pwd
!ls
fulltrain=read_csv('./cs-training.csv', index_col=0)
test=read_csv('./cs-test.csv', index_col=0)

dataSets=[fulltrain,test]

/home/devel/handson-ml/notebook_louis
Intro-Jupyter.ipynb
Intro-Pandas.ipynb
Untitled.ipynb
Untitled1.ipynb
Untitled2.ipynb
bigmlm_first_test_give_me_credit.ipynb
cs-test.csv
cs-training.csv
custom_features.ipynb.txt
custom_features_guillaume.ipynb
guillaume_features_and_predict.ipynb
my_dir
new_features.ipynb
new_features_and_bigml_give_me_credit.ipynb
predictions
testset_dataframe.csv
trainingSet_dataframe.csv


## Correcting the values in monthly income
Set NaN to 0 in the monthly income column

In [18]:

for df in [fulltrain, test]:
    df['MonthlyIncome'].fillna(0,inplace=True)
fulltrain.head(10)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
6,0,0.213179,74,0,0.375607,3500.0,3,0,1,0,1.0
7,0,0.305682,57,0,5710.000000,0.0,8,0,3,0,0.0
8,0,0.754464,39,0,0.209940,3500.0,8,0,0,0,0.0
9,0,0.116951,27,0,46.000000,0.0,2,0,0,0,NaN
10,0,0.189169,57,0,0.606291,23684.0,9,0,4,0,2.0


## Creating a new column for monthly debt.

In [19]:
for df in [fulltrain, test]:
    df.insert(6,"MonthlyDebt",0)
fulltrain.head(10)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,0,7,0,1,0,0.0
6,0,0.213179,74,0,0.375607,3500.0,0,3,0,1,0,1.0
7,0,0.305682,57,0,5710.000000,0.0,0,8,0,3,0,0.0
8,0,0.754464,39,0,0.209940,3500.0,0,8,0,0,0,0.0
9,0,0.116951,27,0,46.000000,0.0,0,2,0,0,0,NaN
10,0,0.189169,57,0,0.606291,23684.0,0,9,0,4,0,2.0


The function calculate the monthly debt.
If the income is zero, we take the monthly debt from the debt ratio.

In [20]:
def monthlyDebtCalc(row):
    if row['MonthlyIncome'] == 0:
        row['MonthlyDebt']=row['DebtRatio']
    else:
        row['MonthlyDebt']=row['DebtRatio']*row['MonthlyIncome']
    return row

fulltrain

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,0,7,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
149996,0,0.040674,74,0,0.225131,2100.0,0,4,0,1,0,0.0
149997,0,0.299745,44,0,0.716562,5584.0,0,4,0,1,0,2.0
149998,0,0.246044,58,0,3870.000000,0.0,0,18,0,1,0,0.0
149999,0,0.000000,30,0,0.000000,5716.0,0,4,0,0,0,0.0


The following block might take some time

In [21]:
for df in [fulltrain, test]:
    tmp=df[['DebtRatio','MonthlyDebt','MonthlyIncome']].apply(monthlyDebtCalc,axis=1)
    df['MonthlyDebt']=tmp['MonthlyDebt']

fulltrain['MonthlyDebt'].head(10)

1      7323.197016
2       316.878123
3       258.914887
4       118.963951
5      1584.975094
6      1314.624392
7      5710.000000
8       734.790059
9        46.000000
10    14359.393699
Name: MonthlyDebt, dtype: float64

# Create New column disposable income

In [22]:
for df in [fulltrain, test]:
    df.insert(7,"DisposableIncome",0)
fulltrain.head(10)


,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,DisposableIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,7323.197016,0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,316.878123,0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,258.914887,0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,118.963951,0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,1584.975094,0,7,0,1,0,0.0
6,0,0.213179,74,0,0.375607,3500.0,1314.624392,0,3,0,1,0,1.0
7,0,0.305682,57,0,5710.000000,0.0,5710.000000,0,8,0,3,0,0.0
8,0,0.754464,39,0,0.209940,3500.0,734.790059,0,8,0,0,0,0.0
9,0,0.116951,27,0,46.000000,0.0,46.000000,0,2,0,0,0,NaN
10,0,0.189169,57,0,0.606291,23684.0,14359.393699,0,9,0,4,0,2.0


In [23]:
def disposableIncomeCalc(row):
    if row['MonthlyIncome'] == 0:
        row['DisposableIncome']= 0
    else:
        row['DisposableIncome']=row['MonthlyIncome']-row['MonthlyDebt']
    return row

fulltrain

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,DisposableIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,7323.197016,0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,316.878123,0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,258.914887,0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,118.963951,0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,1584.975094,0,7,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149996,0,0.040674,74,0,0.225131,2100.0,472.774869,0,4,0,1,0,0.0
149997,0,0.299745,44,0,0.716562,5584.0,4001.283436,0,4,0,1,0,2.0
149998,0,0.246044,58,0,3870.000000,0.0,3870.000000,0,18,0,1,0,0.0
149999,0,0.000000,30,0,0.000000,5716.0,0.000000,0,4,0,0,0,0.0


In [24]:
for df in [fulltrain, test]:
    tmp=df[['MonthlyDebt','MonthlyIncome','DisposableIncome']].apply(disposableIncomeCalc,axis=1)
    df['DisposableIncome']=tmp['DisposableIncome']
    
fulltrain['DisposableIncome'].head(10)

1      1796.802984
2      2283.121877
3      2783.085113
4      3181.036049
5     62003.024906
6      2185.375608
7         0.000000
8      2765.209940
9         0.000000
10     9324.606301
Name: DisposableIncome, dtype: float64

## Correcting the values in number of dependents
Set NaN to 0 in the Number Of Dependents column 

In [25]:
for df in [fulltrain, test]:
    df['NumberOfDependents'].fillna(0,inplace=True)
fulltrain.head(10)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,DisposableIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,7323.197016,1796.802984,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,316.878123,2283.121877,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,258.914887,2783.085113,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,118.963951,3181.036049,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,1584.975094,62003.024906,7,0,1,0,0.0
6,0,0.213179,74,0,0.375607,3500.0,1314.624392,2185.375608,3,0,1,0,1.0
7,0,0.305682,57,0,5710.000000,0.0,5710.000000,0.000000,8,0,3,0,0.0
8,0,0.754464,39,0,0.209940,3500.0,734.790059,2765.209940,8,0,0,0,0.0
9,0,0.116951,27,0,46.000000,0.0,46.000000,0.000000,2,0,0,0,0.0
10,0,0.189169,57,0,0.606291,23684.0,14359.393699,9324.606301,9,0,4,0,2.0


# Disposable Income / dept +1


Create new colomn BalancedIncomePerDependent

In [26]:
for df in [fulltrain, test]:
    df.insert(8,"BalancedIncomePerDependent",0)
fulltrain.head(10)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,DisposableIncome,BalancedIncomePerDependent,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,7323.197016,1796.802984,0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,316.878123,2283.121877,0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,258.914887,2783.085113,0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,118.963951,3181.036049,0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,1584.975094,62003.024906,0,7,0,1,0,0.0
6,0,0.213179,74,0,0.375607,3500.0,1314.624392,2185.375608,0,3,0,1,0,1.0
7,0,0.305682,57,0,5710.000000,0.0,5710.000000,0.000000,0,8,0,3,0,0.0
8,0,0.754464,39,0,0.209940,3500.0,734.790059,2765.209940,0,8,0,0,0,0.0
9,0,0.116951,27,0,46.000000,0.0,46.000000,0.000000,0,2,0,0,0,0.0
10,0,0.189169,57,0,0.606291,23684.0,14359.393699,9324.606301,0,9,0,4,0,2.0


In [27]:
for df in [fulltrain, test]:
    df['BalancedIncomePerDependent']=df['DisposableIncome']/ (df['NumberOfDependents']+1)
    

fulltrain['BalancedIncomePerDependent'].head(10)

1       598.934328
2      1141.560939
3      2783.085113
4      3181.036049
5     62003.024906
6      1092.687804
7         0.000000
8      2765.209940
9         0.000000
10     3108.202100
Name: BalancedIncomePerDependent, dtype: float64

# Create new colomn BalancedIncomePerDependent 

Create new colomn  BalancedIncome  MontlyIncome/ (NumberOfDependents +1)

In [28]:
for df in [fulltrain, test]:
    df.insert(8,"BalancedIncome",0)
fulltrain.head(10)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,DisposableIncome,BalancedIncome,BalancedIncomePerDependent,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,7323.197016,1796.802984,0,598.934328,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,316.878123,2283.121877,0,1141.560939,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,258.914887,2783.085113,0,2783.085113,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,118.963951,3181.036049,0,3181.036049,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,1584.975094,62003.024906,0,62003.024906,7,0,1,0,0.0
6,0,0.213179,74,0,0.375607,3500.0,1314.624392,2185.375608,0,1092.687804,3,0,1,0,1.0
7,0,0.305682,57,0,5710.000000,0.0,5710.000000,0.000000,0,0.000000,8,0,3,0,0.0
8,0,0.754464,39,0,0.209940,3500.0,734.790059,2765.209940,0,2765.209940,8,0,0,0,0.0
9,0,0.116951,27,0,46.000000,0.0,46.000000,0.000000,0,0.000000,2,0,0,0,0.0
10,0,0.189169,57,0,0.606291,23684.0,14359.393699,9324.606301,0,3108.202100,9,0,4,0,2.0


In [29]:
for df in [fulltrain, test]:
    df['BalancedIncome']=df['MonthlyIncome']/ (df['NumberOfDependents']+1)
    

fulltrain['BalancedIncomePerDependent'].head(10)

1       598.934328
2      1141.560939
3      2783.085113
4      3181.036049
5     62003.024906
6      1092.687804
7         0.000000
8      2765.209940
9         0.000000
10     3108.202100
Name: BalancedIncomePerDependent, dtype: float64

# Weighted sum of late payments

In [30]:
for df in [fulltrain, test]:
    df.insert(13,"WeightedOfLatePayment",0)
fulltrain.head(10)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,DisposableIncome,BalancedIncome,BalancedIncomePerDependent,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,WeightedOfLatePayment,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,7323.197016,1796.802984,598.934328,598.934328,13,0,6,0,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,316.878123,2283.121877,1141.560939,1141.560939,4,0,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,258.914887,2783.085113,2783.085113,2783.085113,2,1,0,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,118.963951,3181.036049,3181.036049,3181.036049,5,0,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,1584.975094,62003.024906,62003.024906,62003.024906,7,0,1,0,0,0.0
6,0,0.213179,74,0,0.375607,3500.0,1314.624392,2185.375608,1092.687804,1092.687804,3,0,1,0,0,1.0
7,0,0.305682,57,0,5710.000000,0.0,5710.000000,0.000000,0.000000,0.000000,8,0,3,0,0,0.0
8,0,0.754464,39,0,0.209940,3500.0,734.790059,2765.209940,2765.209940,2765.209940,8,0,0,0,0,0.0
9,0,0.116951,27,0,46.000000,0.0,46.000000,0.000000,0.000000,0.000000,2,0,0,0,0,0.0
10,0,0.189169,57,0,0.606291,23684.0,14359.393699,9324.606301,3108.202100,3108.202100,9,0,4,0,0,2.0


In [31]:
for df in [fulltrain, test]:
    df['WeightedOfLatePayment']=   (df['NumberOfTime30-59DaysPastDueNotWorse']*1) + (df['NumberOfTime60-89DaysPastDueNotWorse']*2) + (df['NumberOfTimes90DaysLate']*3)

# Replace the outlier of DebtRatio by 0.3

In [32]:
def debtRatioCalc(row):
    if row['MonthlyIncome'] == 0:
        row['DebtRatio']= 0.3
        
    return row

In [33]:
for df in [fulltrain, test]:
    tmp=df[['MonthlyIncome','DebtRatio']].apply(debtRatioCalc,axis=1)
    df['DebtRatio']=tmp['DebtRatio']
    
fulltrain['DebtRatio'].head(10)

1     0.802982
2     0.121876
3     0.085113
4     0.036050
5     0.024926
6     0.375607
7     0.300000
8     0.209940
9     0.300000
10    0.606291
Name: DebtRatio, dtype: float64

# Rearrange columns

We test 6 different configurations

In [35]:
fulltrain1 = fulltrain[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]
test1 = test[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]


In [36]:
fulltrain2 = fulltrain[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncomePerDependent', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]
test2 = test[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncomePerDependent', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]



In [72]:
fulltrain3 = fulltrain[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]
test3 = test[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]
export_csv = test3.to_csv (r'validation3.csv',  header=True)


In [38]:
fulltrain4 = fulltrain[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncomePerDependent', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]
test4 = test[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncomePerDependent', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]


In [39]:
fulltrain5 = fulltrain[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]
test5 = test[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines',  'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs']]


In [40]:
fulltrain6 = fulltrain[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'NumberOfDependents', 'SeriousDlqin2yrs']]
test6 = test[['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate',  'NumberOfDependents', 'SeriousDlqin2yrs']]


# BigML API call

On each configuration:
- we make a split with sklearn
- we export in csv trains and tests set obtained

- We train 6 ensemble with the different files prepare

In [42]:
api = BigML()

In [44]:
from sklearn.model_selection import train_test_split
trainingSet1, testSet1 = train_test_split(fulltrain1, test_size=0.2)

export_csv = testSet1.to_csv (r'testset_dataframe1.csv',  header=True) 
export_csv = trainingSet1.to_csv (r'trainingSet_dataframe1.csv',  header=True)

In [45]:
from sklearn.model_selection import train_test_split
trainingSet2, testSet2 = train_test_split(fulltrain2, test_size=0.2)

export_csv = testSet2.to_csv (r'testset_dataframe2.csv',  header=True) 
export_csv = trainingSet2.to_csv (r'trainingSet_dataframe2.csv',  header=True)

In [46]:
from sklearn.model_selection import train_test_split
trainingSet3, testSet3 = train_test_split(fulltrain3, test_size=0.2)

export_csv = testSet3.to_csv (r'testset_dataframe3.csv',  header=True) 
export_csv = trainingSet3.to_csv (r'trainingSet_dataframe3.csv',  header=True)

In [47]:
from sklearn.model_selection import train_test_split
trainingSet4, testSet4 = train_test_split(fulltrain4, test_size=0.2)

export_csv = testSet4.to_csv (r'testset_dataframe4.csv',  header=True) 
export_csv = trainingSet4.to_csv (r'trainingSet_dataframe4.csv',  header=True)

In [48]:
from sklearn.model_selection import train_test_split
trainingSet5, testSet5 = train_test_split(fulltrain5, test_size=0.2)

export_csv = testSet5.to_csv (r'testset_dataframe5.csv',  header=True) 
export_csv = trainingSet5.to_csv (r'trainingSet_dataframe5.csv',  header=True)

In [49]:
from sklearn.model_selection import train_test_split
trainingSet6, testSet6 = train_test_split(fulltrain6, test_size=0.2)

export_csv = testSet6.to_csv (r'testset_dataframe6.csv',  header=True) 
export_csv = trainingSet6.to_csv (r'trainingSet_dataframe6.csv',  header=True)

Train the different ensemble on BigML with training data

In [50]:
source1 = api.create_source('trainingSet_dataframe1.csv')
# waiting for the source to be finished. Results will be stored in `source`
api.ok(source1)
# step 3: creating a dataset from the previously created `source`
dataset1 = api.create_dataset(source1)
# waiting for the dataset to be finished
api.ok(dataset1) 
# step 5: creating an Ensemble
ensemble1 = api.create_ensemble(dataset1)
# waiting for the ensemble to be finished
api.ok(ensemble1)

True

In [51]:
source2 = api.create_source('trainingSet_dataframe2.csv')
# waiting for the source to be finished. Results will be stored in `source`
api.ok(source2)
# step 3: creating a dataset from the previously created `source`
dataset2 = api.create_dataset(source2)
# waiting for the dataset to be finished
api.ok(dataset2) 
# step 5: creating an Ensemble
ensemble2 = api.create_ensemble(dataset2)
# waiting for the ensemble to be finished
api.ok(ensemble2)

True

In [52]:
source3 = api.create_source('trainingSet_dataframe3.csv')
# waiting for the source to be finished. Results will be stored in `source`
api.ok(source3)
# step 3: creating a dataset from the previously created `source`
dataset3 = api.create_dataset(source3)
# waiting for the dataset to be finished
api.ok(dataset3) 
# step 5: creating an Ensemble
ensemble3 = api.create_ensemble(dataset3)
# waiting for the ensemble to be finished
api.ok(ensemble3)

True

In [53]:
source4 = api.create_source('trainingSet_dataframe4.csv')
# waiting for the source to be finished. Results will be stored in `source`
api.ok(source4)
# step 3: creating a dataset from the previously created `source`
dataset4 = api.create_dataset(source4)
# waiting for the dataset to be finished
api.ok(dataset4) 
# step 5: creating an Ensemble
ensemble4 = api.create_ensemble(dataset4)
# waiting for the ensemble to be finished
api.ok(ensemble4)

True

In [54]:
source5 = api.create_source('trainingSet_dataframe5.csv')
# waiting for the source to be finished. Results will be stored in `source`
api.ok(source5)
# step 3: creating a dataset from the previously created `source`
dataset5 = api.create_dataset(source5)
# waiting for the dataset to be finished
api.ok(dataset5) 
# step 5: creating an Ensemble
ensemble5 = api.create_ensemble(dataset5)
# waiting for the ensemble to be finished
api.ok(ensemble5)

True

In [55]:
source6 = api.create_source('trainingSet_dataframe6.csv')
# waiting for the source to be finished. Results will be stored in `source`
api.ok(source6)
# step 3: creating a dataset from the previously created `source`
dataset6 = api.create_dataset(source6)
# waiting for the dataset to be finished
api.ok(dataset6) 
# step 5: creating an Ensemble
ensemble6 = api.create_ensemble(dataset6)
# waiting for the ensemble to be finished
api.ok(ensemble6)

True

Make predictions on test data

In [57]:
# step 6: creating a source from the data in your local "data/test_iris.csv" file
test_source1 = api.create_source("testset_dataframe1.csv")
# waiting for the source to be finished. Results will be stored in `source`
api.ok(test_source1)
# step 8: creating a dataset from the previously created `source`
test_dataset1 = api.create_dataset(test_source1)
# waiting for the dataset to be finished
api.ok(test_dataset1)
# step 10: creating a batch prediction
batch_prediction1 = api.create_batch_prediction(ensemble1, test_dataset1,
                                               {"all_fields": True})
# waiting for the batch_prediction to be finished
api.ok(batch_prediction1)
# downloading the results to your computer
api.download_batch_prediction(batch_prediction1,
                              filename='predictions/my_predictions1.csv')

'predictions/my_predictions1.csv'

In [58]:
# step 6: creating a source from the data in your local "data/test_iris.csv" file
test_source2 = api.create_source("testset_dataframe2.csv")
# waiting for the source to be finished. Results will be stored in `source`
api.ok(test_source2)
# step 8: creating a dataset from the previously created `source`
test_dataset2 = api.create_dataset(test_source2)
# waiting for the dataset to be finished
api.ok(test_dataset2)
# step 10: creating a batch prediction
batch_prediction2 = api.create_batch_prediction(ensemble2, test_dataset2,
                                               {"all_fields": True})
# waiting for the batch_prediction to be finished
api.ok(batch_prediction2)
# downloading the results to your computer
api.download_batch_prediction(batch_prediction2,
                              filename='predictions/my_predictions2.csv')

'predictions/my_predictions2.csv'

In [59]:
# step 6: creating a source from the data in your local "data/test_iris.csv" file
test_source3 = api.create_source("testset_dataframe3.csv")
# waiting for the source to be finished. Results will be stored in `source`
api.ok(test_source3)
# step 8: creating a dataset from the previously created `source`
test_dataset3 = api.create_dataset(test_source3)
# waiting for the dataset to be finished
api.ok(test_dataset3)
# step 10: creating a batch prediction
batch_prediction3 = api.create_batch_prediction(ensemble3, test_dataset3,
                                               {"all_fields": True})
# waiting for the batch_prediction to be finished
api.ok(batch_prediction3)
# downloading the results to your computer
api.download_batch_prediction(batch_prediction3,
                              filename='predictions/my_predictions3.csv')

'predictions/my_predictions3.csv'

In [60]:
# step 6: creating a source from the data in your local "data/test_iris.csv" file
test_source4 = api.create_source("testset_dataframe4.csv")
# waiting for the source to be finished. Results will be stored in `source`
api.ok(test_source4)
# step 8: creating a dataset from the previously created `source`
test_dataset4 = api.create_dataset(test_source4)
# waiting for the dataset to be finished
api.ok(test_dataset4)
# step 10: creating a batch prediction
batch_prediction4 = api.create_batch_prediction(ensemble4, test_dataset4,
                                               {"all_fields": True})
# waiting for the batch_prediction to be finished
api.ok(batch_prediction4)
# downloading the results to your computer
api.download_batch_prediction(batch_prediction4,
                              filename='predictions/my_predictions4.csv')

'predictions/my_predictions4.csv'

In [61]:
# step 6: creating a source from the data in your local "data/test_iris.csv" file
test_source5 = api.create_source("testset_dataframe5.csv")
# waiting for the source to be finished. Results will be stored in `source`
api.ok(test_source5)
# step 8: creating a dataset from the previously created `source`
test_dataset5 = api.create_dataset(test_source5)
# waiting for the dataset to be finished
api.ok(test_dataset5)
# step 10: creating a batch prediction
batch_prediction5 = api.create_batch_prediction(ensemble5, test_dataset5,
                                               {"all_fields": True})
# waiting for the batch_prediction to be finished
api.ok(batch_prediction5)
# downloading the results to your computer
api.download_batch_prediction(batch_prediction5,
                              filename='predictions/my_predictions5.csv')

'predictions/my_predictions5.csv'

In [62]:
# step 6: creating a source from the data in your local "data/test_iris.csv" file
test_source6 = api.create_source("testset_dataframe6.csv")
# waiting for the source to be finished. Results will be stored in `source`
api.ok(test_source6)
# step 8: creating a dataset from the previously created `source`
test_dataset6 = api.create_dataset(test_source6)
# waiting for the dataset to be finished
api.ok(test_dataset6)
# step 10: creating a batch prediction
batch_prediction6 = api.create_batch_prediction(ensemble6, test_dataset6,
                                               {"all_fields": True})
# waiting for the batch_prediction to be finished
api.ok(batch_prediction6)
# downloading the results to your computer
api.download_batch_prediction(batch_prediction6,
                              filename='predictions/my_predictions6.csv')

'predictions/my_predictions6.csv'

Perform an evaluation of the differents sets

In [64]:
test_dataset1 = api.create_dataset(test_source1)
api.ok(test_dataset1)
evaluation1 = api.create_evaluation(ensemble1, test_dataset1)
api.ok(evaluation1)
evaluation1

{'code': 200,
 'resource': 'evaluation/5da9cb7be47684746800222c',
 'location': 'https://bigml.io/andromeda/evaluation/5da9cb7be47684746800222c',
 'object': {'boosted_ensemble': False,
  'category': 0,
  'code': 200,
  'combiner': None,
  'configuration': None,
  'configuration_status': False,
  'created': '2019-10-18T14:26:03.010000',
  'creator': 'maudaby',
  'credits': 0.0,
  'dataset': 'dataset/5da9cb6be476847468002229',
  'dataset_status': True,
  'datasets': [],
  'deepnet': '',
  'description': '',
  'ensemble': 'ensemble/5da9c5c75299631c82001fdc',
  'evaluations': None,
  'excluded_fields': [],
  'fields_map': {'000000': '000000',
   '000001': '000001',
   '000002': '000002',
   '000003': '000003',
   '000004': '000004',
   '000005': '000005',
   '000006': '000006',
   '000007': '000007',
   '000008': '000008',
   '000009': '000009',
   '00000a': '00000a',
   '00000b': '00000b',
   '00000c': '00000c',
   '00000d': '00000d',
   '00000e': '00000e',
   '00000f': '00000f'},
  'fusio

In [66]:
test_dataset2 = api.create_dataset(test_source2)
api.ok(test_dataset2)
evaluation2 = api.create_evaluation(ensemble2, test_dataset2)
api.ok(evaluation2)
evaluation2

{'code': 200,
 'resource': 'evaluation/5da9cce2529963202400048e',
 'location': 'https://bigml.io/andromeda/evaluation/5da9cce2529963202400048e',
 'object': {'boosted_ensemble': False,
  'category': 0,
  'code': 200,
  'combiner': None,
  'configuration': None,
  'configuration_status': False,
  'created': '2019-10-18T14:32:02.252000',
  'creator': 'maudaby',
  'credits': 0.0,
  'dataset': 'dataset/5da9cccde476847468002233',
  'dataset_status': True,
  'datasets': [],
  'deepnet': '',
  'description': '',
  'ensemble': 'ensemble/5da9c61d7811dd01370002c3',
  'evaluations': None,
  'excluded_fields': [],
  'fields_map': {'000000': '000000',
   '000001': '000001',
   '000002': '000002',
   '000003': '000003',
   '000004': '000004',
   '000005': '000005',
   '000006': '000006',
   '000007': '000007',
   '000008': '000008',
   '000009': '000009',
   '00000a': '00000a',
   '00000b': '00000b',
   '00000c': '00000c',
   '00000d': '00000d',
   '00000e': '00000e',
   '00000f': '00000f'},
  'fusio

In [67]:
test_dataset3 = api.create_dataset(test_source3)
api.ok(test_dataset3)
evaluation3 = api.create_evaluation(ensemble3, test_dataset3)
api.ok(evaluation3)
evaluation3

{'code': 200,
 'resource': 'evaluation/5da9cdb3e47684746800223d',
 'location': 'https://bigml.io/andromeda/evaluation/5da9cdb3e47684746800223d',
 'object': {'boosted_ensemble': False,
  'category': 0,
  'code': 200,
  'combiner': None,
  'configuration': None,
  'configuration_status': False,
  'created': '2019-10-18T14:35:31.732000',
  'creator': 'maudaby',
  'credits': 0.0,
  'dataset': 'dataset/5da9cd9f5299631c820020bd',
  'dataset_status': True,
  'datasets': [],
  'deepnet': '',
  'description': '',
  'ensemble': 'ensemble/5da9c6725299631c82001fed',
  'evaluations': None,
  'excluded_fields': [],
  'fields_map': {'000000': '000000',
   '000001': '000001',
   '000002': '000002',
   '000003': '000003',
   '000004': '000004',
   '000005': '000005',
   '000006': '000006',
   '000007': '000007',
   '000008': '000008',
   '000009': '000009',
   '00000a': '00000a',
   '00000b': '00000b',
   '00000c': '00000c',
   '00000d': '00000d',
   '00000e': '00000e',
   '00000f': '00000f'},
  'fusio

In [69]:
test_dataset4 = api.create_dataset(test_source4)
api.ok(test_dataset4)
evaluation4 = api.create_evaluation(ensemble4, test_dataset4)
api.ok(evaluation4)
evaluation4

{'code': 200,
 'resource': 'evaluation/5da9ce955299631c820020cc',
 'location': 'https://bigml.io/andromeda/evaluation/5da9ce955299631c820020cc',
 'object': {'boosted_ensemble': False,
  'category': 0,
  'code': 200,
  'combiner': None,
  'configuration': None,
  'configuration_status': False,
  'created': '2019-10-18T14:39:17.487000',
  'creator': 'maudaby',
  'credits': 0.0,
  'dataset': 'dataset/5da9ce0be476847468002240',
  'dataset_status': True,
  'datasets': [],
  'deepnet': '',
  'description': '',
  'ensemble': 'ensemble/5da9c6c95299631c82001ff6',
  'evaluations': None,
  'excluded_fields': [],
  'fields_map': {'000000': '000000',
   '000001': '000001',
   '000002': '000002',
   '000003': '000003',
   '000004': '000004',
   '000005': '000005',
   '000006': '000006',
   '000007': '000007',
   '000008': '000008',
   '000009': '000009',
   '00000a': '00000a',
   '00000b': '00000b',
   '00000c': '00000c'},
  'fusion': '',
  'input_fields': [],
  'linearregression': '',
  'locale': '

In [70]:
test_dataset5 = api.create_dataset(test_source5)
api.ok(test_dataset5)
evaluation5 = api.create_evaluation(ensemble5, test_dataset5)
api.ok(evaluation5)
evaluation5

{'code': 200,
 'resource': 'evaluation/5da9cf067811dd7f2d00205b',
 'location': 'https://bigml.io/andromeda/evaluation/5da9cf067811dd7f2d00205b',
 'object': {'boosted_ensemble': False,
  'category': 0,
  'code': 200,
  'combiner': None,
  'configuration': None,
  'configuration_status': False,
  'created': '2019-10-18T14:41:10.725000',
  'creator': 'maudaby',
  'credits': 0.0,
  'dataset': 'dataset/5da9ceede47684746800224c',
  'dataset_status': True,
  'datasets': [],
  'deepnet': '',
  'description': '',
  'ensemble': 'ensemble/5da9c717e476847468002189',
  'evaluations': None,
  'excluded_fields': [],
  'fields_map': {'000000': '000000',
   '000001': '000001',
   '000002': '000002',
   '000003': '000003',
   '000004': '000004',
   '000005': '000005',
   '000006': '000006',
   '000007': '000007',
   '000008': '000008',
   '000009': '000009',
   '00000a': '00000a',
   '00000b': '00000b',
   '00000c': '00000c'},
  'fusion': '',
  'input_fields': [],
  'linearregression': '',
  'locale': '

In [71]:
test_dataset6 = api.create_dataset(test_source6)
api.ok(test_dataset6)
evaluation6 = api.create_evaluation(ensemble6, test_dataset6)
api.ok(evaluation6)
evaluation6

{'code': 200,
 'resource': 'evaluation/5dac5c2d5299631c820039e7',
 'location': 'https://bigml.io/andromeda/evaluation/5dac5c2d5299631c820039e7',
 'object': {'boosted_ensemble': False,
  'category': 0,
  'code': 200,
  'combiner': None,
  'configuration': None,
  'configuration_status': False,
  'created': '2019-10-20T13:07:57.722000',
  'creator': 'maudaby',
  'credits': 0.0,
  'dataset': 'dataset/5dac5c187811dd7f2d0038d1',
  'dataset_status': True,
  'datasets': [],
  'deepnet': '',
  'description': '',
  'ensemble': 'ensemble/5da9c7985299631c82002016',
  'evaluations': None,
  'excluded_fields': [],
  'fields_map': {'000000': '000000',
   '000001': '000001',
   '000002': '000002',
   '000003': '000003',
   '000004': '000004',
   '000005': '000005',
   '000006': '000006',
   '000007': '000007',
   '000008': '000008',
   '000009': '000009',
   '00000a': '00000a',
   '00000b': '00000b',
   '00000c': '00000c',
   '00000d': '00000d',
   '00000e': '00000e'},
  'fusion': '',
  'input_fields

We keep the best number : 3 to perform the prediction on the validation data

In [74]:

# step 6: creating a source from the data in your local "data/test_iris.csv" file
validation_source3 = api.create_source("validation3.csv")
# waiting for the source to be finished. Results will be stored in `source`
api.ok(validation_source3)
# step 8: creating a dataset from the previously created `source`
validation_dataset3 = api.create_dataset(validation_source3)
# waiting for the dataset to be finished
api.ok(validation_dataset3)
# step 10: creating a batch prediction
batch_prediction3 = api.create_batch_prediction(ensemble3, validation_dataset3,
                                               {"all_fields": True,
                                                "header": True,
                                                "confidence": True,
                                                "probabilities": True})
# waiting for the batch_prediction to be finished
api.ok(batch_prediction3)
# downloading the results to your computer
api.download_batch_prediction(batch_prediction3,
                              filename='predictions/my_predictions3.csv')

'predictions/my_predictions3.csv'

We recover the prediction

In [7]:
prediction=read_csv('predictions/my_predictions3.csv',index_col=False)
print(list(prediction))

['field1', 'RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs', 'SeriousDlqin2yrs.1', 'confidence', '0 probability', '1 probability']


We rename the columns for rendering

In [8]:
kaggle_prediction=DataFrame()
kaggle_prediction['Id']=prediction['field1']
kaggle_prediction['Probability']=prediction['1 probability']

We check the output

In [9]:
kaggle_prediction.head()

,Id,Probability
0,1,0.05203
1,2,0.03826
2,3,0.00986
3,4,0.09709
4,5,0.13328


Render preparation and sending for evaluation by Kaggle

In [10]:
kaggle_prediction_file="kaggleprediction_3.csv"
kaggle_prediction.to_csv(kaggle_prediction_file,index=False)
kaggle.api.competition_submit(kaggle_prediction_file, "Prediction Bigml", "GiveMeSomeCredit")

100%|██████████| 1.65M/1.65M [00:05<00:00, 333kB/s] 


Successfully submitted to Give Me Some Credit

Maud SONET